In [57]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import os
from torchvision.io import read_image
from PIL import Image
import pandas as pd



niftypath = "/mnt/c/Users/Patrick/Documents/MPHYS_DATA_NIFTY"
croppath = "/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED"

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
print(test_data)

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

NameError: name 'training_data' is not defined

In [117]:
idx = 0
transform = ToTensor()

class CustomImageDataset():
    def __init__(self, croppath, transform=None, target_transfrom=None):
        #self.img_labels = pd.read_csv("/mnt/c/Users/Patrick/Documents/NSCLC Radiomics Lung1.clinical-version3-Oct 2019(1).csv")
        self.img_labels = pd.read_csv("/mnt/c/Users/Patrick/Documents/cancerdata.csv")
        #print(self.img_labels)
        self.img_dir = croppath
        self.transform=transform
        #print(self.transform)
        self.target_transform=target_transfrom

    def __len__ (self):
        return len(self.img_labels)

    def __getitem__ (self, idx):
        img_path = os.path.join(self.img_dir, f"{self.img_labels.iloc[idx, 0]}-RTSTRUCT.nii")
        print(img_path)
        image = sitk.ReadImage(img_path)
        array = sitk.GetArrayFromImage(image)
        tensor = torch.from_numpy(array)
        time_to_death = self.img_labels.iloc[idx,8]
        dead_status = self.img_labels.iloc[idx,9]
        if time_to_death < 1.5*365 and dead_status == 1:
            label=1
        else:
            label=0
        print(label)
        return(label)
dataset = CustomImageDataset(croppath, ToTensor(), None)
#print(len(dataset))

trainset, valset, testset = torch.utils.data.random_split(dataset, [14,3,3])
count=0
#print(trainset.__getitem__(0))



/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-014-RTSTRUCT.nii
0
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-003-RTSTRUCT.nii
1
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-018-RTSTRUCT.nii
0
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-011-RTSTRUCT.nii
1
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-002-RTSTRUCT.nii
1
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-015-RTSTRUCT.nii
0
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-017-RTSTRUCT.nii
1
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-008-RTSTRUCT.nii
1
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-007-RTSTRUCT.nii
1
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-006-RTSTRUCT.nii
1
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-012-RTSTRUCT.nii
1
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-005-RTSTRUCT.nii
1
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-010-RTSTRUCT.nii
0
/mnt/c/Users/Patrick/Docu

In [52]:
xyz = dataset.__getitem__(0)

/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-001-CT.nii


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
z,x,y = np.argwhere(xyz)
ax.scatter(x,y,z,c=z,alpha = 1)
plt.show()

In [118]:
train_dataloader = DataLoader(trainset, batch_size=4, shuffle=True)
train_features = next(iter(train_dataloader))
test_dataloader = DataLoader(testset, batch_size=4, shuffle=True)
test_features = next(iter(train_dataloader))
val_dataloader = DataLoader(valset, batch_size=4, shuffle=True)
val_features = next(iter(val_dataloader))


/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-005-RTSTRUCT.nii
1
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-008-RTSTRUCT.nii
1
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-018-RTSTRUCT.nii
0
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-003-RTSTRUCT.nii
1
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-012-RTSTRUCT.nii
1
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-003-RTSTRUCT.nii
1
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-018-RTSTRUCT.nii
0
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-010-RTSTRUCT.nii
0
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-020-RTSTRUCT.nii
1
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-013-RTSTRUCT.nii
0
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-001-RTSTRUCT.nii
0


In [119]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
model = NeuralNetwork().to(device)
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")



cpu
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Epoch 1
-------------------------------
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-014-RTSTRUCT.nii
0
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-005-RTSTRUCT.nii
1
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-008-RTSTRUCT.nii
1
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-002-RTSTRUCT.nii
1


ValueError: too many values to unpack (expected 2)

In [233]:
flatten = nn.Flatten()
#x = np.array(tensor[0])
#x = torch.tensor(x, dtype=torch.float32)
#print(x)
#print(x.shape)
flat_tensor = tensor[0].flatten(start_dim=0, end_dim=-1)

layer1 = nn.Linear(in_features=172*172*172, out_features=20)
hidden1 = layer1(flat_tensor)
print(hidden1)
hidden1 = nn.ReLU()(hidden1)
print(hidden1)

tensor([-0.0059,  0.0072,  0.0827,  0.0241, -0.1065,  0.1820, -0.0950,  0.0086,
         0.0079,  0.1463, -0.0107,  0.0735, -0.1773, -0.0520, -0.1743,  0.0215,
        -0.0589, -0.0503, -0.0424, -0.0243], grad_fn=<AddBackward0>)
tensor([0.0000, 0.0072, 0.0827, 0.0241, 0.0000, 0.1820, 0.0000, 0.0086, 0.0079,
        0.1463, 0.0000, 0.0735, 0.0000, 0.0000, 0.0000, 0.0215, 0.0000, 0.0000,
        0.0000, 0.0000], grad_fn=<ReluBackward0>)
